In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

In [2]:
data = [
    ("hello", "<start> bonjour <end>"),
    ("how are you", "<start> comment ça va <end>"),
    ("I am fine", "<start> je vais bien <end>"),
    ("what is your name", "<start> comment tu t'appelles <end>"),
    ("my name is", "<start> je m'appelle <end>"),
    ("thank you", "<start> merci <end>"),
    ("goodbye", "<start> au revoir <end>")
]

In [3]:
english_texts, french_texts = zip(*data)
def tokenize(texts):
    tokenizer = Tokenizer(filters='')
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, padding='post')
    return padded_sequences, tokenizer

In [4]:
english_sequences, english_tokenizer = tokenize(english_texts)
french_sequences, french_tokenizer = tokenize(french_texts)
english_vocab_size = len(english_tokenizer.word_index) + 1
french_vocab_size = len(french_tokenizer.word_index) + 1

In [5]:
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(english_vocab_size, 64)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(64, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

In [6]:
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(french_vocab_size, 64)(decoder_inputs)
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(french_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [7]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [8]:
decoder_input_data = french_sequences[:, :-1]
decoder_target_data = french_sequences[:, 1:]
decoder_target_data = np.expand_dims(decoder_target_data, -1)
model.fit([english_sequences, decoder_input_data], decoder_target_data, batch_size=2, epochs=500)
encoder_model = Model(encoder_inputs, encoder_states)

Epoch 1/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 2.7667
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.7390 
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.7050 
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.6407  
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.5980 
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.4709  
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 2.2651
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.2581  
Epoch 9/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.1809  
Epoch 10/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.0627 
Epoch 11/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 2.0390
Epoch 12/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.0861  
Epoch 13/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.0246 
Epoch 14/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 2.0578 
Epoch 15/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.8554 
Epoch 16/500

In [9]:
decoder_state_input_h = Input(shape=(64,))
decoder_state_input_c = Input(shape=(64,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [10]:
def translate_sentence(input_sentence):
    input_sequence = english_tokenizer.texts_to_sequences([input_sentence])
    input_sequence = pad_sequences(input_sequence, maxlen=english_sequences.shape[1], padding='post')
    states_value = encoder_model.predict(input_sequence)
    target_sequence = np.zeros((1, 1))
    target_sequence[0, 0] = french_tokenizer.word_index['<start>']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_sequence] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = french_tokenizer.index_word.get(sampled_token_index, '')
        if sampled_word == '<end>' or len(decoded_sentence) > 20:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word
        target_sequence = np.zeros((1, 1))
        target_sequence[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()

In [11]:
print(translate_sentence("hello"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
bonjour
